In [1]:
from pymongo import MongoClient

def connect_to_prod_mongo():
    #mongo_host = "5.9.1.218"
    mongo_host = "138.201.51.230"
    mongo_port = 27017
    #mongo_db = "dataroot-master"
    mongo_db = "dataroot_nlp"
    #mongo_user = "new-main-worker"
    #mongo_password = "VX4xB,.i(12r{2X|N7~y}02]9533xbpA"
    connection = MongoClient(mongo_host, mongo_port)
    db = connection[mongo_db]
    #db.authenticate(mongo_user, mongo_password)
    return db

In [2]:
db = connect_to_prod_mongo()

In [3]:
db.collection_names(include_system_collections=False)

[u'parsedHtml', u'test', u'companies']

In [43]:
a = db.parsedHtml.find_one()

In [49]:
[i['body']['link'] for i in a['pages']]

[u'http://careers.htc.com/career/HTC-global/',
 u'http://investors.htc.com/phoenix.zhtml?c=148697&p=irol-irhome',
 u'http://www.htc.com/us/about/',
 u'http://www.htc.com/us/smartphones-b/',
 u'http://www.htc.com/us/pressroom/',
 u'http://www.htc.com/us/tablets/nexus-9/',
 u'http://www.htc.com/us/about/device-security/',
 u'http://www.htc.com/us/']

In [94]:
t=[i['body']['text'] for i in a['pages']]

In [118]:
a['pages'][0]['body']['text']

u'\nGlobal \nCareers at HTC \nAbout HTC \nWhy HTC \nOur People \nApply Now \nCareers at HTC \nCareers at HTC \nAbout HTC \n\n\n\nAbout HTC \n\nHow do we define success \n\nHTC Core Values \n\nHTC Charity \nWhy HTC \n\n\n\nFacilities \n\nCompensations &amp; Benefits \n\nCareer Development \n\nOur People \nApply Now \n\n\n\nGlobal Openings \n\nSummer Internship \n\nGlobal \nCareers at HTC \n\n\nthe pursuit of brilliance\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;\nApply Now \nThe place to develop your career \n\n\nPeople are at the core of our innovations. It\u2019s our talented employees of diverse cultures that have taken HTC to its leading position. Right now, we\u2019re looking for new talents who dare to dream, are full of passion and innovative ideas to join our force. Together, we can take HTC to the next level. \nAbout HTC \n\n\nWe are dedicated to enabling a customized user experience based on the belief that each mobile device needs to fit its owner, and not the other way

In [86]:
#print used linkes
c=0
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        c+=1
        print page['body']['link']
print c

http://careers.htc.com/career/HTC-global/
http://investors.htc.com/phoenix.zhtml?c=148697&p=irol-irhome
http://www.htc.com/us/about/
http://www.htc.com/us/smartphones-b/
http://www.htc.com/us/pressroom/
http://www.htc.com/us/tablets/nexus-9/
http://www.htc.com/us/about/device-security/
http://www.htc.com/us/
http://www.apple.com/newsroom/
http://investor.apple.com
http://www.apple.com/apple-events/
http://www.apple.com/pr/
http://www.apple.com/jobs/us/
http://www.apple.com/
http://www.apple.com/contact/
http://www.apple.com/about/
http://company.nokia.com/en/contact
http://company.nokia.com/en/people-planet
http://www.nokia.com/en_int
http://global.blackberry.com/en/company/about-us/corporate-responsibility.html
http://global.blackberry.com/en/company/careers.html
http://global.blackberry.com/en/company/investors.html
http://global.blackberry.com/en/company.html
http://global.blackberry.com/en/home.html
http://www.wearebosch.com/index.en.html
http://www.bosch.com/en/com/products_servic

In [130]:
import numpy as np
import nltk
from nltk.metrics import *
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

In [124]:
def removePunctuation(text):
    p = re.compile('[^a-zA-Z0-9_ ]')
    return p.sub('', text.lower()).strip()

In [134]:
with open('terrier-stop.txt') as f:
    stop_words = [i.strip() for i in f.readlines()]
print "We have %d stop words" % len(stop_words)

We have 733 stop words


In [112]:
#site - words
documents = []
for doc in db.parsedHtml.find():
    documents += [filter(lambda x: x!= u'', 
                         map(lambda x: removePunctuation(x), #x.strip(), 
                             sum([i['body']['text'].split('\n') for i in doc['pages']],[])
                            )
                     )
                 ]

print "sites - %d" % len(documents)

sites - 8


In [165]:
#page - words
documents = []
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        documents += [filter(lambda x: x!= u'', 
                             map(lambda x: removePunctuation(x), #x.strip(), 
                                 page['body']['text'].split('\n')
                                )
                          )
                     ]
        
print "pages - %d" % len(documents)

pages - 62


In [172]:
#sentence - words
documents = []
for doc in db.parsedHtml.find():
    for page in doc['pages']:
        documents += filter(lambda x: x!= u'',
                            map(lambda x: [removePunctuation(x)], #x.strip(), 
                                page['body']['text'].split('\n')
                               )
                        )
        
print "sentences - %d" % len(documents)

sentences - 18254


In [57]:
#filter code
for i, doc in enumerate(documents):
    documents[i] = [ d for d in doc if not('{' in d or '}' in d or d[-2:]==');')]

In [144]:
documents = [doc for doc in documents if len(doc) >10]

In [173]:
#with open('site - words.txt', 'w+') as f:
#with open('page - words.txt', 'w+') as f:
with open('sentence - words.txt', 'w+') as f:
    for doc in documents:
        f.write(' '.join(doc).encode('utf-8')+'\n')

In [175]:
#cut out stop words ans stem
ps = PorterStemmer()
for i, el in enumerate(documents):
    for j, doc in enumerate(el):
        documents[i][j] = ' '.join(
                                    map(lambda x: ps.stem(x),
                                        filter(lambda x: x not in stop_words, 
                                          word_tokenize(doc))
                                       )
                                   )
    documents[i] = filter(lambda x: x!=u'', documents[i])

In [178]:
documents[0:20]

[[u'global'],
 [u'career htc'],
 [u'htc'],
 [u'htc'],
 [u'peopl'],
 [u'appli now'],
 [u'career htc'],
 [u'career htc'],
 [u'htc'],
 [u'htc'],
 [u'defin success'],
 [u'htc core valu'],
 [u'htc chariti'],
 [u'htc'],
 [u'facil'],
 [u'compens amp benefit'],
 [u'career develop'],
 [u'peopl'],
 [u'appli now'],
 [u'global open']]